IMPORTING LIBRARIES

In [ ]:
import os
os.chdir("D:/Final ADM/Extracted dataset/")

import shutil
import glob
import cars_utils
import matplotlib.pyplot as plt
from functools import partial
from PIL import Image
from multiprocessing import Pool
%matplotlib inline

CHOOSING 10 CLASSES OF CARS

In [ ]:
cars = {
    "honda_civic_1998": ["honda_civic_1997", "honda_civic_1998"], # available "honda_civic_1999"
    "honda_accord_1997": ["honda_accord_1996", "honda_accord_1997"],
    "ford_f150_2006": ["ford_f150_2005", "ford_f150_2006", "ford_f150_2007"],
    "chevrolet_silverado_2004": ["chevrolet_silverado_2003", "chevrolet_silverado_2004"], # available "chevrolet_silverado_2005"
    "toyota_camry_2014": ["toyota_camry_2012", "toyota_camry_2013", "toyota_camry_2014", "toyota_camry_le_2012", "toyota_camry_le_2013", "toyota_camry_le_2014", "toyota_camry_se_2012", "toyota_camry_se_2013", "toyota_camry_xle_2012", "toyota_camry_xle_2013"],
    "nissan_altima_2014": ["nissan_altima_2013", "nissan_altima_2014", "nissan_altima_2015"],
    "toyota_corolla_2013": ["toyota_corolla_2011", "toyota_corolla_2012", "toyota_corolla_2013", "toyota_corolla_ce_2012", "toyota_corolla_le_2012", "toyota_corolla_le_2013", "toyota_corolla_s_2011", "toyota_corolla_s_2012"],
    "dodge_ram_2001": ["dodge_ram_1500_2000", "dodge_ram_1500_2001", "dodge_ram_1500_1999", "dodge_ram_1500_1998", "dodge_ram_1500_1997", "dodge_ram_1500_1996", "dodge_ram_1500_1995"],
    "gmc_sierra_2012": ["gmc_sierra_1500_2007", "gmc_sierra_1500_2008", "gmc_sierra_1500_2009", "gmc_sierra_1500_2010", "gmc_sierra_1500_2011", "gmc_sierra_1500_2012", "gmc_sierra_1500_2013", "gmc_sierra_2500_2007", "gmc_sierra_2500_2008", "gmc_sierra_2500_2009", "gmc_sierra_2500_2010", "gmc_sierra_2500_2011", "gmc_sierra_2500_2012", "gmc_sierra_2500_2013"],
    "chevrolet_impala_2008": ["chevrolet_impala_2007", "chevrolet_impala_2008", "chevrolet_impala_2009"]
}

sETTING THE PATH

In [ ]:
full_dataset_path = "D:/Final ADM/Extracted dataset"
stolen_cars_path = "D:/Final ADM/Most_stolen_cars"

REMOVE THE ALREADY EXISITING FOLDERS

In [ ]:
if os.path.exists(stolen_cars_path):
    shutil.rmtree(stolen_cars_path)
else:
    os.makedirs(stolen_cars_path)

In [ ]:
for directory, car_list in cars.items():
    print("Creating", directory)
    car_directory_name = os.path.join(stolen_cars_path, directory)
    os.makedirs(car_directory_name)
    for car in car_list:
        path = os.path.join(full_dataset_path, car, "")
        files = glob.glob(path + '*.jpg')
        for file in files:
            shutil.copy(file, car_directory_name)

cars_utils.display_images(stolen_cars_path)


In [ ]:
#Removing .png and other corrupt files
import os

dir_name = "D:/Final ADM/Most_stolen_cars"
test = os.listdir(dir_name)

for item in test:
    if item.endswith(".png"):
        os.remove(os.path.join(dir_name, item))
        
for item in test:
    if item.endswith(".jpeg"):
        os.remove(os.path.join(dir_name, item))

In [ ]:
#checking various files used in the data
if __name__ == '__main__':
    pool = Pool()
    image_list = glob.glob(stolen_cars_path + "/*/*")
    pool.map(cars_utils.check_image, image_list)
    pool.close()

print('Done.')

In [ ]:
#Using Pygal and galplot to visualize the data
import pygal 
from IPython.display import display, HTML
#Create function to display interactive plotting
base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

def galplot(chart):
    rendered_chart = chart.render(is_unicode=True)
    plot_html = base_html.format(rendered_chart=rendered_chart)
    display(HTML(plot_html))


In [ ]:
line_chart = pygal.Bar(height=300)
line_chart.title = 'Stolen Car Class Distribution'
for o in os.listdir(stolen_cars_path):
    line_chart.add(o, len(os.listdir(os.path.join(stolen_cars_path, o))))
    line_chart.render_to_file('D:/Final ADM/class_distribution.svg')
galplot(line_chart)

In [ ]:
for root, dirs, files in os.walk(stolen_cars_path):
    level = root.replace(os.getcwd(), '').count(os.sep)
    print('{0}{1}/'.format('    ' * level, os.path.basename(root)))
    for f in files[:2]:
        print('{0}{1}'.format('    ' * (level + 1), f))
    if level is not 0:
        print('{0}{1}'.format('    ' * (level + 1), "..."))


In [ ]:
import math
import re
import sys

In [ ]:
#Train and Test Set Variables
train_val_test_ratio = (.8,.05,.15) # 80/5/15 Data Split
test_folder = 'D:/Final ADM/proc_data/test/'
train_folder = 'D:/Final ADM/proc_data/train/'
val_folder = 'D:/Final ADM/proc_data/val/'

In [ ]:
file_names = os.listdir('D:/Final ADM/Most_stolen_cars')

In [ ]:
#Remove Existing Folders
for folder in [test_folder, train_folder, val_folder]:
    if os.path.exists(folder) and os.path.isdir(folder):
        shutil.rmtree(folder)

In [ ]:
for category in file_names:
    os.makedirs(test_folder + category)
    os.makedirs(train_folder + category)
    os.makedirs(val_folder + category)

In [ ]:
for idx, category in enumerate(file_names):
    file_list = os.listdir(stolen_cars_path + '/' + category)
    
    train_ratio = math.floor(len(file_list) * train_val_test_ratio[0])
    val_ratio = math.floor(len(file_list) * train_val_test_ratio[1])
    train_list = file_list[:train_ratio]
    val_list = file_list[train_ratio:train_ratio + val_ratio]
    test_list = file_list[train_ratio + val_ratio:]
    for i, file in enumerate(train_list):
        shutil.copy(stolen_cars_path + '/' + category + '/' + file, train_folder + '/' + category + '/' + file)
        sys.stdout.write('Moving %s train images to category folder %s' % (len(train_list), category))  
        sys.stdout.write('\n')
    for i,file in enumerate(val_list):
        shutil.copy(stolen_cars_path + '/' + category + '/' + file, val_folder + '/' + category + '/' + file)
        sys.stdout.write('Moving %s validation images to category folder %s' % (len(val_list), category))                   
        sys.stdout.write('\n')
    for i, file in enumerate(test_list):
        shutil.copy(stolen_cars_path + '/' + category + '/' + file, test_folder + '/' + category + '/' + file)
        sys.stdout.write('Moving %s test images to category folder %s' % (len(test_list), category))
        sys.stdout.write('\n')
    
print("Done.")

In [ ]:
import random
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

#Select a random image and follow the next step
datagen = ImageDataGenerator(rotation_range=45, 
                             width_shift_range=0.2, 
                             height_shift_range=0.2, 
                             zoom_range=0.3, 
                             vertical_flip=True,
                             horizontal_flip=True, 
                             fill_mode="nearest")

In [ ]:
file_list = glob.glob("D:/Final ADM/proc_data/test/*/*") 
img_path = random.choice(file_list)
img = load_img(img_path)
car_class = img_path.split("/")[1]
plt.imshow(img)
plt.axis("off") 
plt.title("Original " + car_class, fontsize=16)

img = img_to_array(img)
img = img.reshape((1,) + img.shape)

In [ ]:
n_augmentations = 4   #"""Change this to 3"""
plt.figure(figsize=(15, 6))    
i = 0
for batch in datagen.flow(img, 
                          batch_size=1, 
                          seed=21):
    
    plt.subplot(2, int(np.ceil(n_augmentations * 1. / 2)), i + 1)
    plt.imshow(array_to_img(batch[0]))
    plt.axis("off")
    plt.suptitle("Augmented " + car_class, fontsize=16)    
    
    i += 1
    if i >= n_augmentations:
        break

In [ ]:
def data_augment(data_dir):
    list_of_images = os.listdir(data_dir)
    datagen = ImageDataGenerator(rotation_range=45, 
        horizontal_flip=True, 
        fill_mode="nearest")
    for img_name in list_of_images:
        tmp_img_name = os.path.join(data_dir, img_name)
        img = load_img(tmp_img_name)
        img = img_to_array(img)
        img = img.reshape((1,) + img.shape)

        batch = datagen.flow(img, 
            batch_size=1, 
            seed=21,
            save_to_dir=data_dir, 
            save_prefix=img_name.split(".jpg")[0] + "augmented", 
            save_format="jpg")

        batch.next()

In [ ]:
classes_to_augment = [
    "toyota_camry_2014",
    "nissan_altima_2014",
    "toyota_corolla_2013",
    "gmc_sierra_2012",
    ]

for class_names in classes_to_augment:
    print("Currently Augmenting:", class_names)
    data_dir = os.path.join(train_folder, class_names)
    data_augment(data_dir)

In [ ]:
def resize_image(file, size=299):
    print(file)
    img = Image.open(file)
    img = img.resize((size,size))
    img.save(file)

#Resize Images
if __name__ == '__main__':
    pool = Pool()
    image_list = glob.glob(train_folder + "/*/*")
    [resize_image(x) for x in image_list]
    func = partial(resize_image, size=299)
    pool.close()

In [ ]:
cars_utils.display_images(train_folder)

In [ ]:
line_chart = pygal.Bar(height=300)
line_chart.title = 'Most Stolen Car Training Class Distribution'
for o in os.listdir(train_folder):
    line_chart.add(o, len(os.listdir(os.path.join(train_folder, o))))
galplot(line_chart)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input, decode_predictions

WIDTH=224
HEIGHT=224
BATCH_SIZE=16
test_dir = 'D:/Final ADM/proc_data/test/'
train_dir = 'D:/Final ADM/proc_data/train/'
val_dir = 'D:/Final ADM/proc_data/val/'

#Train DataSet Generator with Augmentation
print("\nTraining Data Set")
train_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_flow = train_generator.flow_from_directory(
    train_dir,
    target_size=(HEIGHT, WIDTH),
    batch_size = BATCH_SIZE
)

#Validation DataSet Generator with Augmentation
print("\nValidation Data Set")
val_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
val_flow = val_generator.flow_from_directory(
    val_dir,
    target_size=(HEIGHT, WIDTH),
    batch_size = BATCH_SIZE
)

#Test DataSet Generator with Augmentation
print("\nTest Data Set")
test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_flow = test_generator.flow_from_directory(
    test_dir,
    target_size=(HEIGHT, WIDTH),
    batch_size = BATCH_SIZE
)

In [ ]:
# Initialize VGG16 with transfer learning
base_model = applications.VGG16(weights='imagenet', 
                                include_top=False, 
                                input_shape=(WIDTH, HEIGHT,3))

# add a global spatial average pooling layer
x = base_model.output

x = GlobalAveragePooling2D()(x)
# and a dense layer
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_flow.class_indices), activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional VGG16 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=optimizers.Adam(lr=0.001), metrics=['accuracy', 'top_k_categorical_accuracy'], loss='categorical_crossentropy')
model.summary()

In [ ]:
import math
top_layers_file_path="top_layers.vgg16.hdf5"

checkpoint = ModelCheckpoint(top_layers_file_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
tb = TensorBoard(log_dir='D:/Final ADM/LOGS/logs', batch_size=val_flow.batch_size, write_graph=True, update_freq='batch')
early = EarlyStopping(monitor="loss", mode="min", patience=5)
csv_logger = CSVLogger('D:/Final ADM/LOGS/logs/vgg16-log.csv', append=True)

history = model.fit_generator(train_flow, 
                              epochs=170, 
                              verbose=1,
                              validation_data=val_flow,
                              validation_steps=math.ceil(val_flow.samples/val_flow.batch_size),
                              steps_per_epoch=math.ceil(train_flow.samples/train_flow.batch_size),
                              callbacks=[checkpoint, early, tb, csv_logger])

In [ ]:
model.load_weights(top_layers_file_path)
loss, acc, top_5 = model.evaluate_generator(
    test_flow,
    verbose = True,
    steps=math.ceil(test_flow.samples/test_flow.batch_size))
print("Loss: ", loss)
print("Acc: ", acc)
print("Top 5: ", top_5)

In [ ]:
label = [k for k,v in train_flow.class_indices.items()]
with open('vgg16-labels.txt', 'w+') as file:
    file.write("\n".join(label))

In [ ]:
from keras.preprocessing import image
import numpy as np
import glob
import random

file_list = glob.glob("D:/Final ADM/proc_data/test/*/*")
img_path = random.choice(file_list)
img_cat = os.path.split(os.path.dirname(img_path))[1]
print("Image Category: ", img_cat)
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print("Raw Predictions: ", preds)

top_x = 3
top_args = preds[0].argsort()[-top_x:][::-1]
preds_label = [label[p] for p in top_args]
print("\nTop " + str(top_x) + " confidence: " + " ".join(map(str, sorted(preds[0])[-top_x:][::-1])))
print("Top " + str(top_x) + " labels: " + " ".join(map(str, preds_label)))

In [ ]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

input_model_path = top_layers_file_path
output_model_name = "top_layers.vgg16.pb"
output_model_dir = "tf_model"

K.set_learning_phase(0)
sess = K.get_session()

test_model = models.load_model(input_model_path)
orig_output_node_names = [node.op.name for node in test_model.outputs]

constant_graph = graph_util.convert_variables_to_constants(
    sess,
    sess.graph.as_graph_def(),
    orig_output_node_names)
graph_io.write_graph(
    constant_graph,
    output_model_dir,
    output_model_name,
    as_text=False)

In [ ]:
model = load_model(top_layers_file_path)

# unfreeze all layers
for layer in model.layers:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=optimizers.Adam(lr=0.0001), metrics=['accuracy', 'top_k_categorical_accuracy'], loss='categorical_crossentropy')
model.summary()

In [ ]:
#Start Training top layers
all_nodes_file_path="all_layers.vgg16.hdf5"
checkpoint = ModelCheckpoint(all_nodes_file_path, monitor='loss', verbose=1, save_best_only=True, mode='min')

train_flow.reset()
val_flow.reset()
history = model.fit_generator(train_flow, 
                              epochs=170, 
                              verbose=1,
                              validation_data = val_flow,
                              validation_steps= math.ceil(val_flow.samples/val_flow.batch_size),
                              steps_per_epoch = math.ceil(train_flow.samples/train_flow.batch_size),
                              callbacks = [checkpoint, early, tb, csv_logger])

In [ ]:
from keras.models import load_model #importing the model
model = load_model('top_layers.vgg16.hdf5')

In [ ]:

    from keras.preprocessing.image import ImageDataGenerator
    from keras.applications.vgg16 import preprocess_input
    #Test DataSet Generator with Augmentation
    test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
    test_flow = test_generator.flow_from_directory(
        'D:/Final ADM/proc_data/test',
        shuffle=False,
        target_size=(224, 224),
        batch_size = 16
    )

In [ ]:
    import math
    import numpy as np
    predictions = model.predict_generator(
        test_flow,
        verbose=1,
        steps=math.ceil(test_flow.samples/test_flow.batch_size))
    predicted_classes = np.argmax(predictions, axis=1) 
    
    true_classes = test_flow.classes
    class_labels = list(test_flow.class_indices.keys())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import scikitplot as skplt

In [ ]:
[print(k, ":", v) for k,v in enumerate(class_labels)]
true_map_classes = [class_labels[x] for x in true_classes]
predicted_map_classes = [class_labels[x] for x in predicted_classes]

In [ ]:
skplt.metrics.plot_confusion_matrix(
        true_map_classes, 
        predicted_map_classes,
        labels=class_labels,
        x_tick_rotation=90,
        figsize=(40, 12))

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(
    true_classes,
    predicted_classes,
    target_names=class_labels)
print(report)

In [ ]:
skplt.metrics.plot_precision_recall(
    true_classes,
    predictions,
    figsize=(12,12))

In [ ]:
skplt.metrics.plot_roc(
    true_classes,
    predictions,
    figsize = (12,12))
